In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import yfinance as yf
import matplotlib.pyplot as plt
import os
from keras import backend as K
import tensorflow as tf

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Clear any existing Keras sessions
K.clear_session()

# Define stock tickers
techassets = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'ORCL', 'META', 'TSLA', 'NVDA']
bankingassets = ['BRK-B', 'JPM', 'V', 'BAC', 'AXP', 'GS', 'BLK', 'STT']
healthcareassets = ['LLY', 'JNJ', 'ABT', 'MDT', 'UNH', 'PFE', 'CVS', 'MRK']
all_assets = techassets + bankingassets + healthcareassets

# Define date range
start_date = '2014-01-01'
end_date = '2025-02-23'  # Set to current date

# Fetch stock data using yfinance
stock_data = {}
failed_downloads = []

for ticker in all_assets:
    try:
        stock = yf.download(ticker, start=start_date, end=end_date)
        if len(stock) > 0:
            stock_data[ticker] = stock
            print(f'Successfully downloaded data for {ticker}, {len(stock)} rows')
        else:
            print(f'No data available for {ticker}')
            failed_downloads.append(ticker)
    except Exception as e:
        print(f'Failed to download data for {ticker}: {e}')
        failed_downloads.append(ticker)

# Remove failed tickers from the list
all_assets = [ticker for ticker in all_assets if ticker not in failed_downloads]

# Function to prepare time series data with sequence length of 30 days
def prepare_time_series_data(data, sequence_length=30):
    # Extract close prices
    close_prices = data['Close'].values.reshape(-1, 1)
    
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)
    
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(scaled_data[i+sequence_length])
    
    return np.array(X), np.array(y), scaler

# Function to create a fresh model
def create_fresh_model(input_shape):
    # Clear any existing Keras session
    K.clear_session()
    
    # Create a new model with fresh weights
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Dictionary to store models, predictions, and metrics
results = {}

# Process each ticker individually
# for ticker in all_assets[:9]:  # Start with just the first 8 tickers for testing
for ticker in all_assets:       # Uncomment to process all assets
    print(f"\nProcessing {ticker}:")
    
    # Remove any existing model files to prevent loading from them
    if os.path.exists(f'best_model_{ticker}.keras'):
        os.remove(f'best_model_{ticker}.keras')
    
    # Prepare data
    data = stock_data[ticker]
    X, y, scaler = prepare_time_series_data(data)
    
    # Split into train and test sets (90/10)
    train_size = int(len(X) * 0.9)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Testing data shape: {X_test.shape}")
    
    # Create a fresh model for each ticker
    model = create_fresh_model((X_train.shape[1], X_train.shape[2]))
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'best_model_{ticker}.keras', monitor='val_loss', save_best_only=True)
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,  # Reduced for faster execution
        batch_size=36,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    # Make predictions
    predictions = model.predict(X_test)
    
    # Inverse transform to get actual price values
    y_test_actual = scaler.inverse_transform(y_test)
    predictions_actual = scaler.inverse_transform(predictions)
    
    # Calculate metrics
    mse = np.mean((predictions_actual - y_test_actual) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions_actual - y_test_actual))
    
    print(f"Results for {ticker}:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    
    # Store results
    results[ticker] = {
        'model': model,
        'history': history,
        'predictions': predictions_actual,
        'actual': y_test_actual,
        'metrics': {
            'mse': mse,
            'rmse': rmse,
            'mae': mae
        },
        'scaler': scaler
    }
    
    # Visualize predictions vs actual
    plt.figure(figsize=(12, 6))
    plt.title(f'{ticker} Stock Price Prediction')
    plt.plot(y_test_actual, label='Actual Price')
    plt.plot(predictions_actual, label='Predicted Price')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig(f'{ticker}_prediction.png')
    plt.close()

# Function to make future predictions
def predict_future(model, last_sequence, scaler, days_to_predict=30):
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(days_to_predict):
        # Get prediction for next day
        next_day_pred = model.predict(current_sequence.reshape(1, current_sequence.shape[0], current_sequence.shape[1]))
        
        # Add prediction to future predictions list
        future_predictions.append(next_day_pred[0, 0])
        
        # Update sequence by removing first value and adding the prediction
        current_sequence = np.append(current_sequence[1:], next_day_pred, axis=0)
    
    # Scale predictions back to original price
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)
    
    return future_predictions

# Generate future predictions for all tickers
for ticker in all_assets:  # Use the same subset as used in training
    print(f"\nGenerating future predictions for {ticker}")
    
    # Get the last sequence from the data
    data = stock_data[ticker]
    sequence_length = 30
    close_prices = data['Close'].values.reshape(-1, 1)
    scaler = results[ticker]['scaler']
    scaled_data = scaler.transform(close_prices)
    last_sequence = scaled_data[-sequence_length:]
    
    # Predict future prices
    future_days = 30
    future_predictions = predict_future(results[ticker]['model'], last_sequence, scaler, days_to_predict=future_days)
    
    # Generate future dates
    last_date = data.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)
    
    # Plot actual and future predicted prices
    plt.figure(figsize=(12, 6))
    plt.title(f'{ticker} Stock Price with Future Prediction')
    
    # Plot historical data (last 90 days)
    historical_days = 90
    historical_dates = data.index[-historical_days:]
    historical_prices = data['Close'].values[-historical_days:]
    plt.plot(historical_dates, historical_prices, label='Historical Price', color='blue')
    
    # Plot future predictions
    plt.plot(future_dates, future_predictions, label='Future Predictions', color='red', linestyle='--')
    
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{ticker}_future_prediction.png')
    plt.close()
    
    print(f"\nFuture prediction for {ticker} completed")
    print("Future prices for the next 30 days:")
    for date, price in zip(future_dates, future_predictions):
        print(f"{date.date()}: ${price[0]:.2f}")

print("\nScript execution completed.")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Successfully downloaded data for AAPL, 2802 rows


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Successfully downloaded data for MSFT, 2802 rows
Successfully downloaded data for GOOG, 2802 rows


...

Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 18s 96ms/step - loss: 0.0955 - val_loss: 0.0019
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - loss: 0.0023 - val_loss: 0.0027
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.0021 - val_loss: 0.0034
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0025 - val_loss: 0.0037
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - loss: 0.0020 - val_loss: 0.0043
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - loss: 0.0020 - val_loss: 0.0227
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 0.0026 - val_loss: 9.7348e-04
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - loss: 0.0021 - val_loss: 9.5007e-04
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0020 - val_loss: 4.8761e-04
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - loss: 0.0020 - val_loss: 3.0441e-04
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - loss: 0.0020 - val_loss: 2.4727e-04
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\lynch\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step - loss: 0.1469 - val_loss: 0.0405
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 0.0030 - val_loss: 0.0366
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0028 - val_loss: 0.0364
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0027 - val_loss: 0.0362
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 0.0026 - val_loss: 0.0346
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - loss: 0.0025 - val_loss: 0.0281
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 0.0025 - val_loss: 0.0223
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - loss: 0.0025 - val_loss: 0.0166
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - loss: 0.0025 - val_loss: 0.0096
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - loss: 0.0025 - val_loss: 0.0057
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0025 - val_loss: 0.0015
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0

C:\Users\lynch\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - loss: 0.0920 - val_loss: 0.2900
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.0015 - val_loss: 0.2691
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0015 - val_loss: 0.2487
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 0.0016 - val_loss: 0.2339
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - loss: 0.0017 - val_loss: 0.2430
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - loss: 0.0017 - val_loss: 0.1795
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0017 - val_loss: 0.1336
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0017 - val_loss: 0.0809
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0017 - val_loss: 0.0368
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.0017 - val_loss: 0.0134
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0017 - val_loss: 0.0035
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.